In [ ]:
import dateutil.parser as dateparser
from pymongo import MongoClient
import utils.const as const
import langid

In [ ]:
client = MongoClient()
langid.set_languages(['gu','en','hi'])

In [ ]:
db=const.grpId
collection=const.PostCollection

data='lemmatized_data.txt'
count='count.txt'
id_='id.txt'

wordDictFile='word_dictionary.csv'
wordDict={}

In [ ]:
myCollect=client[db][collection]

In [ ]:
#################################ignore
# c=0
# for post in list(myCollect.find()):
#     #replace every time field with str(dateparser.parse(s).timestamp())
#     if 'comments' in post:
#         for comment in post['comments']:
#             if 'created_time' not in comment:
#                 myCollect.update_one({'_id': post['_id'], 'comments': {'$elemMatch': {'_id': comment['_id']}}}, {
#                     "$set": {'comments.$.message' : 'Congratulations',   "comments.$.created_time": "2020-03-13T04:13:03+0000"}
#                 })
#                 c+=1

# print(c)

In [ ]:
for post in list(myCollect.find()):
    #replace every time field with str(dateparser.parse(s).timestamp())
    myCollect.update_one({'_id': post['_id']}, {
            "$set": {'updated_time' : str(dateparser.parse(post['updated_time']).timestamp())}
        })
   
    myCollect.update_one({'_id': post['_id']}, {
            "$set": {'created_time' : str(dateparser.parse(post['created_time']).timestamp())}
        })
    
    if 'comments' in post:
        for comment in post['comments']:
            myCollect.update_one({'_id': post['_id'], 'comments': {'$elemMatch': {'_id': comment['_id']}}}, {
                "$set": {'comments.$.created_time' : str(dateparser.parse(comment['created_time']).timestamp())}
            })
            
            if 'subComments' in comment:
                
                for subComment in comment['subComments']:
                    subComment['created_time']=str(dateparser.parse(subComment['created_time']).timestamp())
                    
                myCollect.update_one({'_id': post['_id'], 'comments': {'$elemMatch': {'_id': comment['_id']}}}, {
                    "$set": {'comments.$.subComments' : comment['subComments']}
                })

# for post in list(myCollect.find()):
#     #replace every time field with str(dateparser.parse(s).timestamp())
#     if 'updated_time'not in post:
#         continue
        
#     t=dateparser.parse(post['updated_time']).timestamp()
    
#     myCollect.update_one({'_id': post['_id']}, {
#             "$set": {'updated_time' : str(t)}
#         })
    
#     if 'comments' in post:
#         for comment in post['comments']:
#             t2=dateparser.parse(comment['created_time']).timestamp()
#             myCollect.update_one({'_id': post['_id'], 'comments': {'$elemMatch': {'_id': comment['_id']}}}, {
#                 "$set": {'comments.$.created_time' : str(t2)}
#             })
            
#             t=min(t,t2)
            
#             if 'subComments' in comment:
                
#                 for subComment in comment['subComments']:
#                     t2=dateparser.parse(subComment['created_time']).timestamp()
#                     subComment['created_time']=str(t2)
#                     t=min(t,t2)
                    
#                 myCollect.update_one({'_id': post['_id'], 'comments': {'$elemMatch': {'_id': comment['_id']}}}, {
#                     "$set": {'comments.$.subComments' : comment['subComments']}
#                 })
    
#     if 'created_time' in post:
#         myCollect.update_one({'_id': post['_id']}, {
#                 "$set": {'created_time' : str(dateparser.parse(post['created_time']).timestamp())}
#             })
#     else:
#         myCollect.update_one({'_id': post['_id']}, {
#                 "$set": {'created_time' : str(t)}
#             })
       

In [ ]:
for post in list(myCollect.find()):
    lang='NA'
    if 'message' in post and len(post['message']):
        lang=langid.classify(post['message'])[0]
        
    myCollect.update_one({'_id': post['_id']}, {
            "$set": {'lang' : lang}
        })
    
    if 'comments' in post:
        for comment in post['comments']:
            lang='NA'
            if 'message' in comment and len(comment['message']):
                lang=langid.classify(comment['message'])[0]
            
            myCollect.update_one({'_id': post['_id'], 'comments': {'$elemMatch': {'_id': comment['_id']}}}, {
                "$set": {'comments.$.lang' : lang}
            })
            
            if 'subComments' in comment:
                
                for subComment in comment['subComments']:
                    lang='NA'
                    if 'message' in subComment and len(subComment['message']):
                        lang=langid.classify(subComment['message'])[0]
                        
                    subComment['lang']=lang
                    
                myCollect.update_one({'_id': post['_id'], 'comments': {'$elemMatch': {'_id': comment['_id']}}}, {
                    "$set": {'comments.$.subComments' : comment['subComments']}
                })